# csv, SQLite を探すサンプル

csv や SQLite から、LLM を利用してクエリをもとにデータを探し出すサンプル

In [ ]:
import os
f"OpenAI API Key is {os.environ['OPENAI_API_KEY']}."


サンプルデータとして、人口推計表をダウンロードする。
https://dashboard.e-stat.go.jp/timeSeriesResult?indicatorCode=0201010000000010000 から「ダウンロード」ボタンをクリックして csv ファイルをダウンロードする。

> 出典：政府統計の総合窓口(e-Stat)（https://www.e-stat.go.jp/）

以下の `file_name` 変数を適当に書き換える。

In [25]:
file_name = '/path/to/TimeSeriesResult_YYYYMMDDHHMMSSF.csv'

## csv からデータを探す。
csv ファイルから目的のデータを探し出すクエリを投げる。

In [26]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
agent = create_csv_agent(OpenAI(), file_name, verbose=True)
agent.run("何年が総人口の最大値を持つ?")



> Entering new AgentExecutor chain...


Thought: 総人口が最も多い年を探します。
Action: python_repl_ast
Action Input: df[df['総人口（総数）【人】'].idxmax()]
Observation: 64
Thought:

 最大値を持つ時点を調べます。
Action: python_repl_ast
Action Input: df.at[df['総人口（総数）【人】'].idxmax(), '時点']
Observation: 2008 年
Thought:

 最大値を持つのは2008年です。
Final Answer: 2008年が総人口の最大値を持つ。

> Finished chain.


'2008年が総人口の最大値を持つ。'

## SQLite からデータを探す。
1. 上で使用した csv ファイルを `pandas.DataFrame` で読み出す。
2. それを `to_sql()` で SQLite に保存する。
3. 保存されたデータベースから欲しいデータを検索する。


In [30]:
import pandas as pd
import sqlite3

df = pd.read_csv(file_name, encoding="utf-8")
with sqlite3.connect("sample.db") as conn:
    df.to_sql('sample', conn, if_exists = 'fail', index = False)


In [32]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

db = SQLDatabase.from_uri('sqlite:///./sample.db')
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=OpenAI(),
    toolkit=toolkit,
    verbose=True
)

agent_executor.run("何年が総人口の最大値を持つ?")




> Entering new AgentExecutor chain...


Action: list_tables_sql_db
Action Input: ""
Observation: sample
Thought:

 I can use the table 'sample' to get the answer to the question.
Action: schema_sql_db
Action Input: "sample"
Observation: 
CREATE TABLE sample (
	"時点" TEXT, 
	"地域コード" INTEGER, 
	"地域" TEXT, 
	"総人口（総数）【人】" INTEGER, 
	"注記" TEXT
)
/*
3 rows from sample table:
時点	地域コード	地域	総人口（総数）【人】	注記
1920 年	0	全国	55963053	総務省統計局「国勢調査」による。
1925 年	0	全国	59736822	総務省統計局「国勢調査」による。
1930 年	0	全国	64450005	総務省統計局「国勢調査」による。
*/
Thought:

 I can use the '時点' and '総人口（総数）【人】' columns to get the answer.
Action: query_checker_sql_db
Action Input: "SELECT 時点, MAX(総人口（総数）【人】) FROM sample"


Observation: 

SELECT 時点, MAX(総人口（総数）【人】) FROM sample;
Thought:

 The query looks correct, so I can execute it.
Action: query_sql_db
Action Input: "SELECT 時点, MAX(総人口（総数）【人】) FROM sample"
Observation: [('2008 年', 128084000)]
Thought:

 I now know the answer to the original question.
Final Answer: 2008年が総人口の最大値を持つ

> Finished chain.


'2008年が総人口の最大値を持つ'